<a href="https://colab.research.google.com/github/jpmartin22/jpmartin22/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/ensemble-core/NdLinear.git
%cd NdLinear

Cloning into 'NdLinear'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 38 (delta 11), reused 22 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 63.20 KiB | 21.07 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/NdLinear


In [4]:

import sys
sys.path.append('/content/NdLinear')

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
from ndlinear import NdLinear


In [5]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 76.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 17.0MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 50.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.50MB/s]


In [10]:
# Standard MLP
class StandardMLP(nn.Module):
    def __init__(self):
        super(StandardMLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        return self.fc2(x)

# MLP with NdLinear
import torch.nn as nn

class NdLinearMLP(nn.Module):
    def __init__(self):
        super(NdLinearMLP, self).__init__()
        self.fc1 = NdLinear((28*28,), (128,))
        self.relu = nn.ReLU()
        self.fc2 = NdLinear((128,), (10,))

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        return self.fc2(x)


In [11]:
def train_model(model, trainloader, criterion, optimizer, device):
    model.to(device)
    model.train()
    total_loss = 0
    for inputs, targets in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(trainloader)

def test_model(model, testloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return 100. * correct / total


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# Training Standard MLP
std_model = StandardMLP()
std_optimizer = optim.Adam(std_model.parameters(), lr=0.001)

# Training NdLinear MLP
nd_model = NdLinearMLP()
nd_optimizer = optim.Adam(nd_model.parameters(), lr=0.001)

print("Training Standard MLP...")
for epoch in range(5):
    loss = train_model(std_model, trainloader, criterion, std_optimizer, device)
    acc = test_model(std_model, testloader, device)
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, Accuracy={acc:.2f}%")

print("\nTraining NdLinear MLP...")
for epoch in range(5):
    loss = train_model(nd_model, trainloader, criterion, nd_optimizer, device)
    acc = test_model(nd_model, testloader, device)
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, Accuracy={acc:.2f}%")


Training Standard MLP...
Epoch 1: Loss=0.3528, Accuracy=94.27%
Epoch 2: Loss=0.1637, Accuracy=95.82%
Epoch 3: Loss=0.1136, Accuracy=96.85%
Epoch 4: Loss=0.0860, Accuracy=97.31%
Epoch 5: Loss=0.0685, Accuracy=97.29%

Training NdLinear MLP...
Epoch 1: Loss=0.3478, Accuracy=94.30%
Epoch 2: Loss=0.1584, Accuracy=96.28%
Epoch 3: Loss=0.1116, Accuracy=97.00%
Epoch 4: Loss=0.0845, Accuracy=97.06%
Epoch 5: Loss=0.0673, Accuracy=97.50%
